In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR

Fazemos a leitura dos dados de todos os meses para coletar e repassar o máximo de informações possíveis para o modelo de previsão.

In [60]:
df_agosto = pd.read_csv('../data/Agosto/Ana Health_Tabela Modelo Previsão Churn - Tabela até 08_23.csv', skiprows=1)
df_julho = pd.read_csv('../data/Julho/Ana Health_Tabela Modelo Previsão Churn - Tabela até 07_23.csv', skiprows=1)
df_junho = pd.read_csv('../data/Junho/Ana Health_Tabela Modelo Previsão Churn - Tabela até 06_23.csv', skiprows=1)
df_novembro = pd.read_csv('../data/Novembro/Ana Health_Tabela Modelo Previsão Churn - Tabela Geral.csv', skiprows=1)
df_outubro = pd.read_csv('../data/Outubro/Ana Health_Tabela Modelo Previsão Churn - Tabela até 10_23.csv', skiprows=1)
df_setembro = pd.read_csv('../data/Setembro/Ana Health_Tabela Modelo Previsão Churn - Tabela até 09_23.csv', skiprows=1)

Aplicamos os tratamentos básicos de limpeza e transformação dos dados em todos os meses.

In [61]:
import script_data_basico
import importlib
importlib.reload(script_data_basico)
tratamento = script_data_basico.tratamento

fim_junho = datetime(2023, 6, 30)
fim_julho = datetime(2023, 7, 31)
fim_agosto = datetime(2023, 8, 31)
fim_setembro = datetime(2023, 9, 30)
fim_outubro = datetime(2023, 10, 31)
fim_novembro = datetime(2023, 11, 30)

df_agosto = tratamento(df_agosto,fim_agosto,df_setembro)
df_julho = tratamento(df_julho, fim_julho, df_agosto)
df_junho = tratamento(df_junho, fim_junho, df_julho)
df_outubro = tratamento(df_outubro, fim_outubro, df_novembro)
df_setembro = tratamento(df_setembro, fim_setembro, df_outubro)

Juntamos os dados de todos os meses em um único dataframe.

In [62]:
df_total = pd.concat([df_junho, df_julho, df_agosto, df_setembro, df_outubro])

Aplicamos o tratamento de dados para o nosso modelo de regressão, temos o df_regr que usaremos para treinar o modelo. E também temos o df_regr_coerencia, que é constituído apenas dos dados das pessoas que ainda estão na empresa, para termos uma espécie de "teste de coerência" que o intuito de verificar se o modelo está prevendo corretamente (se ele prever que a pessoa deveria já ter saído ele estaria errado). 

In [63]:
import script_data_regressao

df_regr, df_regr_coerencia = script_data_regressao.tratamento_regressao(df_total)

In [64]:
# Embaralhamento dos dados para não ficar na ordem de data e não ter viés

df_regr = df_regr.sample(frac=1)
df_regr_coerencia = df_regr_coerencia.sample(frac=1)

### Random Forest Regression

In [81]:
# Divisão dos dados em treino e teste
y = df_regr['Tempo até Sair']
X = df_regr.drop(['Tempo até Sair'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Criação do modelo de regressão
model = RandomForestRegressor(random_state=42)

# Escolhemos os parâmetros que queremos testar no modelo de regressão através do GridSearchCV
param_grid = {
    'n_estimators': [1, 10, 100, 500],
    'max_depth': [None, 3, 5, 10, 50],
    'min_samples_split': [0.001, 0.01, 0.1, 2],
    'min_samples_leaf': [1, 2, 5, 10]
}

# Obtemos o melhor modelo através do GridSearchCV
model = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)

# Treinamos o modelo com os dados de treino
model.fit(X_train, y_train)

# Obtemos as previsões do modelo
y_pred = model.predict(X_test)

# Calculamos a performance do modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))
print("Melhores parâmetros:", model.best_params_)


Mean Squared Error: 7868.9925692095285
Root Mean Squared Error (Erro em dias de atraso): 88.70734225085052
Melhores parâmetros: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 0.001, 'n_estimators': 500}


In [84]:
joblib.dump(model, 'regression_model.joblib')

['regression_model.joblib']

Aqui realizamos um teste para verificar se o modelo está coerente com a realidade.

Para isso, utilizamos o dataset de pessoas que ainda estão ativas para tentar entender o comportamento do modelo ao prever o tempo de cancelamento e comparar com o tempo real que a pessoa está ativa.

In [66]:
# Usando os dados de clientes que ainda estão para comparar com os dados de clientes que sairam apenas para testar o modelo
y = df_regr_coerencia['Tempo até Sair']
X = df_regr_coerencia.drop(['Tempo até Sair'], axis=1)

# Obtemos as previsões do modelo
y_pred = model.predict(X)

# Calculamos a performance do modelo
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))

# Verificar se está errando para mais ou para menos
print('Erro médio:', np.mean(y_pred - y))

Mean Squared Error: 15987.001261835016
Root Mean Squared Error (Erro em dias de atraso): 126.43971394239634
Erro médio: -106.93332324073005


### Linear Regression

In [67]:
# Divisão dos dados em treino e teste
y = df_regr['Tempo até Sair']
X = df_regr.drop(['Tempo até Sair'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)  # Adjust test_size

# Criação do modelo de regressão
model = LinearRegression()

# Parâmetros que queremos testar no modelo de regressão através do GridSearchCV
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, 1, 2, 5, 10],
    'positive': [True, False],

}

# Obtemos o melhor modelo através do GridSearchCV
model = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)

# Treinando o modelo
model.fit(X_train, y_train)

# Obtendo as previsões do modelo
y_pred = model.predict(X_test)

# Calculamos a performance do modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))
print("Melhores parâmetros:", model.best_params_)



Mean Squared Error: 11685.326031623177
Root Mean Squared Error (Erro em dias de atraso): 108.09868653976874
Melhores parâmetros: {'copy_X': True, 'fit_intercept': False, 'n_jobs': None, 'positive': False}


### Support Vector Regression

In [68]:
# Parâmetros que queremos testar no modelo de regressão através do GridSearchCV
param_grid = {
    'C': [0.1, 0.5, 1, 2, 3, 4, 5, 10],
    'epsilon': [0.01, 0.1, 0.5, 1, 2, 3, 4, 5, 10, 50, 100],
    'kernel': ['linear', 'rbf', 'poly']
}

# Criação do modelo de regressão
model = SVR()

# Obtemos o melhor modelo através do GridSearchCV
model = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)

# Treina o modelo
model.fit(X_train, y_train)

# Obtém as previsões do modelo
y_pred = model.predict(X_test)

# Calculamos a performance do modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))
print("Melhores parâmetros:", model.best_params_)

Mean Squared Error: 11632.619156796005
Root Mean Squared Error (Erro em dias de atraso): 107.85462047031645
Melhores parâmetros: {'C': 2, 'epsilon': 50, 'kernel': 'linear'}


### Lasso

In [69]:
# Parâmetros que queremos testar no modelo de regressão através do GridSearchCV
param_grid = {
    'alpha':[0.001,0.01,0.05,0.1,0.5,1, 2, 3, 4, 5, 10],
    "fit_intercept": [True, False],
    "copy_X": [True,False],
    "warm_start": [True, False],
    "selection": ['cyclic', 'random']
}

# Cria modelo
model = Lasso()

# Obtemos o melhor modelo através do GridSearchCV para o modelo de regressão com Lasso
model = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)
model.fit(X_train, y_train)

# Faz as previsões do modelo
y_pred = model.predict(X_test)

# Calculamos a performance do modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))
print("Melhores parâmetros:", model.best_params_)


Mean Squared Error: 12846.508386030006
Root Mean Squared Error (Erro em dias de atraso): 113.3424385922149
Melhores parâmetros: {'alpha': 2, 'copy_X': True, 'fit_intercept': True, 'selection': 'random', 'warm_start': True}


### Ridge

In [70]:
# Cria modelo
model = Ridge()

# Parâmetro que queremos testar no modelo Ridge através do GridSearchCV
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100]
}


# Obtemos o melhor modelo através do GridSearchCV para o modelo
model = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)
model.fit(X_train, y_train)

# Faz as previsões do modelo
y_pred = model.predict(X_test)

# Calculamos a performance do modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))
print("Melhores parâmetros:", model.best_params_)


Mean Squared Error: 12714.51546160036
Root Mean Squared Error (Erro em dias de atraso): 112.75866025099961
Melhores parâmetros: {'alpha': 100}


### Elastic Net

In [71]:
param_grid = {
    'alpha':[0.001,0.01,0.05,0.1,0.5,1],
    'l1_ratio':[0.1, 0.01, 0.5, 0.05, 0.001],
    "fit_intercept": [True, False],
    "copy_X": [True,False],
    "warm_start": [True, False],
    "selection": ['cyclic', 'random']
}

# Cria modelo
model = ElasticNet()

# Obtém o melhor modelo através do GridSearchCV para o modelo ElasticNet
model = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)
model.fit(X_train, y_train)

# Calcula as previsões do modelo
y_pred = model.predict(X_test)

# Calculamos a performance do modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error (Erro em dias de atraso):", np.sqrt(mse))
print("Melhores parâmetros:", model.best_params_)

Mean Squared Error: 12796.600707773283
Root Mean Squared Error (Erro em dias de atraso): 113.12206110115429
Melhores parâmetros: {'alpha': 0.5, 'copy_X': False, 'fit_intercept': True, 'l1_ratio': 0.5, 'selection': 'random', 'warm_start': False}
